In [82]:
# MOD per system
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck') 


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# SAM Edges and Number for Example

frequency bins/edges

In [83]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

Number of frequency bins: 4
  between [0.63, 3.16] 1/yr
          [20.00, 100.00] nHz


Semi-Analytic-Model with default parameters 
* gsmf=GSMF_Schechter, 
* gpf=GPF_Power_Law, 
* gmt=GMT_Power_Law, 
* mmbulge=relations.MMBulge_MM2013

In [84]:
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
print('edges:', sam.edges)

edges: [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ])]


gwb (internal to sam.gwb excluding hc calculation)

In [85]:
fobs_gw_edges = fobs_edges
hard = holo.hardening.Hard_GW
fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges)
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0


# dynamic_binary_number
# gets differential number of binaries per bin-vol per log freq interval
edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))

In [86]:
def gws_by_loops(edges, number, realize, print_test = False, sum=False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)

    """
    
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    if(print_test):
        print('Observer frame frequency centers: ', fc)
        print('Mass edges: ', mt.shape, mt)
        print('Ratio edges:', mr.shape, mr)
        print('Redshift edges:', rz.shape, rz,'\n')

    
    # make strain grid
    hc_grid = np.empty_like(number)

    # for r realizations, make a grid with shape 
    # m, q, z, f, r
    if(utils.isinteger(realize)):
        newshape = hc_grid.shape + (realize,)
        if(print_test):
            print('newshape:', newshape)
        realized_grid = np.empty(newshape)

    # get strain for each bin
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            for z_idx in range(len(rz)):
                cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_mqzf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    
                    if(realize == False):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_mqzf 
                                        * number[m_idx, q_idx, z_idx, f_idx])
                    elif(realize == True):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_mqzf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                    elif(utils.isinteger(realize)):
                        for r_idx in range(realize):
                            realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                                np.sqrt(hc_mqzf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                        hc_grid = realized_grid                
                    else:
                        print("`realize` ({}) must be one of {{True, False, integer}}!"\
                            .format(realize))

                     # print fr, hs, hc^2/dlnf, number, hc
                    if(print_test):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_mqzf))
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_grid[m_idx, q_idx, z_idx, f_idx])
    if(sum):
        # sum over all bins at a given frequency and realization
        hc_grid = np.sqrt(np.sum(hc_grid**2, axis=(0, 1, 2)))

    
    return hc_grid


In [113]:
def gws_by_ndars(edges, number, realize, print_test = False, sum=False):
       
    """ More efficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M-1, Q-1, Z-1, F-1)
        realize = True or False, sum = True: shape is (F-1)
        realize = R, sum = False: shape is  (M-1, Q-1, Z-1, F-1, R)
        realize = R, sum = True: shape is  (F-1, R)

    """

    if(print_test):
        print('INPUTS: edges:', len(edges), # '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M-1, Q-1) we need 
    # mt in shape (M-1, 1) 
    # mr in shape (1, Q-1)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])
    if(print_test):
        print('cmass:', cmass.shape, '\n', cmass)

    # --- Comoving Distances ---
    # to get cdist in shape (Z-1) we need
    # rz in shape (Z-1)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value
    if(print_test):
        print('cdist:', cdist.shape, '\n', cdist)

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z-1, F-1) we need 
    # rz in shape (Z-1, 1) 
    # fc in shape (1, F-1)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])
    if(print_test):
        print('rfreq:', rfreq.shape, '\n', rfreq)

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M-1, Q-1, Z-1, F-1) we need
    # cmass in shape (M-1, Q-1, 1, 1) from (M-1, Q-1)
    # cdist in shape (1, 1, Z-1, 1) from (Z-1)
    # rfreq in shape (1, 1, Z-1, F-1) from (Z-1, F-1)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])
    if(print_test):
        print('hsamp', hsamp.shape, '\n', hsamp)

    # --- Characteristic Strain Squared ---
    # to get characteristic strain in shape (M-1, Q-1, Z-1, F-1) we need
    # hsamp in shape (M-1, Q-1, Z-1, F-1)
    # fc in shape (1, 1, 1, F-1)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:]
                        /df[np.newaxis, np.newaxis, np.newaxis,:])

    # Sample:
    if(realize == False):
        # without sampling, want strain in shape (M-1, Q-1, Z-1, F-1)
        hchar *= number

    if(realize == True):
        # with a single sample, want strain in shape (M-1, Q-1, Z-1, F-1)
        hchar *= np.random.poisson(number)

    if(utils.isinteger(realize)):
        # with R realizations, 
        # to get strain in shape (M-1, Q-1, Z-1, F-1, R) we need
        # hchar in shape(M-1, Q-1, Z-1, F-1, 1)
        # Poisson sample in shape (1, 1, 1, 1, R)
        npois = np.random.poisson(number[...,np.newaxis], size = (number.shape + (realize,)))
        if(print_test):
            print('npois', npois.shape)
        hchar = hchar[...,np.newaxis] * npois


    if(print_test):
        print('hchar', hchar.shape, '\n', hchar)


    if(sum):
        # sum over all bins at a given frequency and realization
        hchar = np.sum(hchar, axis=(0, 1, 2))
        # NOTE I should check what big O time this is,  not sure
        if(print_test):
            print('hchar summed', hchar.shape, '\n', hchar)

    return np.sqrt(hchar)

In [114]:
np.set_printoptions(precision=2)
print('------BY NDARS------')
ndar_hcF = gws_by_ndars(edges, number, realize=False, print_test=True, sum=False)
print('\n------BY LOOPS------')
loop_ncF = gws_by_loops(edges, number, realize=False, print_test=True, sum=False)
np.set_printoptions()

------BY NDARS------
INPUTS: edges: 4 
INPUTS:number: (2, 1, 3, 5) 
 [[[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [8.83e+06 6.46e+05 1.44e+05 5.01e+04 2.78e+04]
   [1.09e+09 8.01e+07 1.79e+07 6.21e+06 3.44e+06]]]


 [[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [1.10e+05 8.05e+03 1.80e+03 6.24e+02 3.46e+02]
   [1.29e+07 9.45e+05 2.11e+05 7.32e+04 4.06e+04]]]] 

cmass: (2, 1) 
 [[2.27e+39]
 [2.27e+40]]
cdist: (3,) 
 [7.33e+25 7.25e+26 6.42e+27]
rfreq: (3, 5) 
 [[1.01e-08 2.01e-08 3.02e-08 4.02e-08 5.28e-08]
 [1.06e-08 2.11e-08 3.17e-08 4.22e-08 5.54e-08]
 [1.55e-08 3.10e-08 4.65e-08 6.20e-08 8.14e-08]]
hsamp (2, 1, 3, 5) 
 [[[[2.92e-19 4.64e-19 6.07e-19 7.36e-19 8.82e-19]
   [3.05e-20 4.84e-20 6.34e-20 7.68e-20 9.21e-20]
   [4.45e-21 7.07e-21 9.26e-21 1.12e-20 1.34e-20]]]


 [[[1.36e-17 2.15e-17 2.82e-17 3.42e-17 4.09e-17]
   [1.41e-18 2.25e-18 2.94e-18 3.57e-18 4.27e-18]
   [2.07e-19 3.28e-19 4.30e-19 5.21e-19 6.24e-19]]]]
hchar (2, 1, 3, 5) 
 [[[[8.53e-38 4.30e-37 1.11e-

# Test Cases

In [115]:
# get GWB spectrum
hc_samF = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False, sum=False)
hc_samF_sum = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False, sum=True)
hc_samT = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=True, sum=False)
hc_sam1 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=1, sum=False)
hc_sam3 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=3, sum=False)
hc_sam3_sum = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=3, sum=True)

hc_ndarF = gws_by_ndars(edges, number, realize=False, sum=False)   
hc_ndarF_sum = np.sqrt(np.sum(hc_ndarF**2, axis=(0, 1, 2))) 
hc_ndarF_sum2 = gws_by_ndars(edges, number, realize=False, sum=True)
hc_ndarT = gws_by_ndars(edges, number, realize=True)
hc_ndar1 = gws_by_ndars(edges, number, realize=1)   
hc_ndar3 = gws_by_ndars(edges, number, realize=3)   
hc_ndar3_sum = np.sqrt(np.sum(hc_ndar3**2, axis=(0, 1, 2)))
        

In [116]:
print(hc_samF == hc_ndarF)
# should be exact same
print('hc by sam:\n', hc_samF)
print('hc by ndar:\n', hc_ndarF)

[[[[ True  True  True  True  True]
   [ True  True  True  True  True]
   [ True  True  True  True  True]]]


 [[[ True  True  True  True  True]
   [ True  True  True  True  True]
   [ True  True  True  True  True]]]]
hc by sam:
 [[[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [9.06e-17 5.50e-17 4.17e-17 3.44e-17 2.87e-17]
   [1.47e-16 8.95e-17 6.79e-17 5.59e-17 4.67e-17]]]


 [[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [4.69e-16 2.85e-16 2.16e-16 1.78e-16 1.49e-16]
   [7.42e-16 4.51e-16 3.42e-16 2.82e-16 2.35e-16]]]]
hc by ndar:
 [[[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [9.06e-17 5.50e-17 4.17e-17 3.44e-17 2.87e-17]
   [1.47e-16 8.95e-17 6.79e-17 5.59e-17 4.67e-17]]]


 [[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
   [4.69e-16 2.85e-16 2.16e-16 1.78e-16 1.49e-16]
   [7.42e-16 4.51e-16 3.42e-16 2.82e-16 2.35e-16]]]]


In [117]:
print(hc_samF_sum == hc_ndarF_sum)
print(hc_samF_sum == hc_ndarF_sum2)
# should be exact same
print('hc sum by sam:\n', hc_samF_sum)
print('hc sum by ndar:\n', hc_ndarF_sum)

[ True  True  True  True  True]
[ True  True  True  True  True]
hc sum by sam:
 [8.95e-16 5.44e-16 4.12e-16 3.40e-16 2.84e-16]
hc sum by ndar:
 [8.95e-16 5.44e-16 4.12e-16 3.40e-16 2.84e-16]


# Scratch

### poisson sampling

In [92]:
print(np.random.poisson(number).shape)
realize=5
newshape = number.shape + (realize,)
# print((number.shape + (5,)).shape)
print(np.random.poisson(number[:,:,:,:,np.newaxis], size=newshape).shape)

(2, 1, 3, 5)
(2, 1, 3, 5, 5)


### comoving distance

In [93]:
red_temp = np.array([1,2,3])
print(holo.cosmo.comoving_distance(red_temp).cgs)

[1.0367e+28 1.6307e+28 2.0040e+28] cm


### chirp mass

In [94]:
# checking out utils.chirp_mass_mtmr
mtotarr = np.array([[1e40, 2e40, 3e40],[1e40, 2e40, 3e40],[1e40, 2e40, 3e40],[1e40, 2e40, 3e40]])
mratarr = np.array([[.1, .1, .1], [.2, .2, .2], [.3, .3, .3], [.4, .4, .4]])
mchiarr = utils.chirp_mass_mtmr(mtotarr, mratarr)
print(mchiarr.shape)
print(mchiarr)

(4, 3)
[[2.2404e+39 4.4808e+39 6.7212e+39]
 [3.0591e+39 6.1183e+39 9.1774e+39]
 [3.5444e+39 7.0888e+39 1.0633e+40]
 [3.8537e+39 7.7075e+39 1.1561e+40]]


In [95]:
mtotarr = np.array([1e40, 2e40])
mratarr = np.array([.1, .2, .3])
mttemp = mtotarr[:, np.newaxis]
mrtemp = mratarr[np.newaxis,:]
print(mttemp)
print(mrtemp)
print(utils._array_args(mttemp, mrtemp))
print(mttemp * mrtemp)
# print(utils.chirp_mass_mtmr(mttemp, mrtemp))

# math function of a (1xN) array and a (Mx1) array produces an NxM array

# we want shape (M x Q), comes from (M x 1) * (1 x Q)
# put mass in form (M x 1), this is just a new view of the same array
mtemp = mtotarr[..., np.newaxis]
print(mtemp.shape)
qtemp = mratarr[np.newaxis, ...]
print(qtemp.shape)
print('mchirp', utils.chirp_mass_mtmr(mtemp,qtemp).shape)
# put rat in form (1 x Q)
# give mass as 

[[1.e+40]
 [2.e+40]]
[[0.1 0.2 0.3]]
[array([[1.e+40],
       [2.e+40]]), array([[0.1, 0.2, 0.3]])]
[[1.e+39 2.e+39 3.e+39]
 [2.e+39 4.e+39 6.e+39]]
(2, 1)
(1, 3)
mchirp (2, 3)


In [96]:
temp1 = np.array([5,5,5])
temp2 = temp1[:,np.newaxis]
temp1[1] = -1
print(np.shares_memory(temp1, temp2))
print(temp2)
print(temp1.base) #??? how is this none? now temp1 is a view of temp2?

True
[[ 5]
 [-1]
 [ 5]]
None


In [97]:
x_arr = np.array([[1,2,3]])
y_arr = np.array([[1],[10],[100],[1000]])
print(x_arr*y_arr)

# similarly, passing utils.chirp_mass_mtmr a (nx)

[[   1    2    3]
 [  10   20   30]
 [ 100  200  300]
 [1000 2000 3000]]


In [98]:
np.set_printoptions(precision = 4)
ndar_hcF = gws_by_ndars(edges, number, realize = False, print_test=True)
np.set_printoptions(precision = None)

INPUTS: edges: 4 
INPUTS:number: (2, 1, 3, 5)
cmass: (2, 1)
cdist: (3,)
rfreq: (3, 5)
hsamp (2, 1, 3, 5)
hsamp (2, 1, 3, 5)
hchar (2, 1, 3, 5)
